##### Store Procedures

In [1]:
import pyodbc
import pandas as pd

In [2]:
# Trusted Connection to SQL Server Default Instance with Exception Handling (invalid DB Name)
try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=MSI;DATABASE=sampleDB;Trusted_Connection=yes;')

    # initialize the cursor object with connection name
    cursor = connection.cursor()
    print()
    print('Successfully Connected to Database')
    print()

except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()
    


Successfully Connected to Database



##### Create SQL Server Stored Procedure

In [4]:
try:
    # If stored procedure already exists, use ALTER PROCEDURE
    # using """ """ This can create a multiple line SQL Query for easier Readability
    sqlCommand = """ 
    CREATE PROCEDURE uspCustDetails (@cusID int, @out nvarchar(max) OUTPUT)
    AS
    SELECT * FROM tblCustomers WHERE id=@cusID
    """
    
    # execute query
    cursor.execute(sqlCommand)
    connection.commit()
    print()
    print('Stored Procedure Successfully Created')
    print()

except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()


Stored Procedure Successfully Created



##### Call SQL Server Stored Procedure that returns result

In [5]:
try:
    # If stored procedure already exists, use ALTER PROCEDURE
    # using """ """ This can create a multiple line SQL Query for easier Readability
    # Define  sqlCommand string to call a stored procedure with parameters
    sqlCommand = """ 
    DECLARE @out nvarchar(max)
    EXEC dbo.uspCustDetails @cusID=?, @out = @out OUTPUT
    """

    # Initialize the parameter with a specific customer ID to pass to the stored procedure
    paramCusId = 2

    # Execute the stored procedure using the SQL command and parameter
    cursor.execute(sqlCommand, paramCusId)
    # Notify that stored procedure results are being printed
    print('[Stored Procedure Call Results...]')

    # Fetch all rows returned by the stored procedure
    rows = cursor.fetchall()
    # Display the fetched results and continue fetching if more result sets are available
    while rows:
        print(rows)  # Print current batch of rows
        if cursor.nextset():  # Check if there is another set of results
            rows = cursor.fetchall()  # Fetch next set of results
        else:
            rows = None  # No more results, exit loop

    print()

except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()

[Stored Procedure Call Results...]
[(2, 'code2', 'firstName2', 'lastName2')]



##### Call SQL Server Stored Procedure that returns no result

In [10]:
try:
    # Create no result Stored Procedure
    sqlCommand = """ 
    CREATE PROCEDURE uspCustDetailsUpdate (@cusID int)
    AS
    UPDATE tblCustomers SET lastName= lastName + ' Update from SP'
    WHERE id=@cusID
    """
    # Execute the stored procedure creation using the SQL command
    cursor.execute(sqlCommand)
    # Commits transaction
    connection.commit()
    print()
    
    # Call stored procedure
    sqlCommand1 = """ 
    EXEC dbo.uspCustDetailsUpdate @cusID=?
    """
    paramCusId = 2
    # Execute the stored procedure using the SQL command and parameter
    cursor.execute(sqlCommand1, paramCusId)
    # Commits transaction
    connection.commit()
    print()
    # Select Query after UPDATE SP Called
    cursor.execute("SELECT * FROM tblCustomers")
    print("[Results after Customer Detail Update Store Procedure Called...]")
    while 1:
        row = cursor.fetchone()
        if not row:
            break
        print(row.id,row.code,row.firstName,row.lastName)
    print()   

except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()



Results after Customer Detail Update Store Procedure Called
1 code1 firstName1 lastName1
2 code2 firstName2 lastName2 Update from SP
3 code3 firstName3 lastName3
4 code4 firstName4 lastName4
5 code5 firstName5 lastName5

